# Importing Libraries

In [1]:
# pytorch
import torch

# huggingface
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Configuration

In [ ]:
class CONFIG:
    device = None
    
    # Model
    base_model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
    model_id = "Waktaverse-Llama-3-KO-8B-Instruct"
    username = "PathFinderKR"
    repo_id = f"{username}/{model_id}"

# Device

In [3]:
def configure_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        num_gpu = torch.cuda.device_count()
        print("> Running on GPU", end=' | ')
        print("Num of GPUs: ", num_gpu)
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
        print("> Running on MPS")
    else:
        device = torch.device("cpu")
        print("> Running on CPU")
    return device

CONFIG.device = configure_device()

Device = cuda:0


# Merge

In [5]:
tokenizer = AutoTokenizer.from_pretrained(CONFIG.base_model_id)
base_model = AutoModelForCausalLM.from_pretrained(
    CONFIG.base_model_id,
    device_map=CONFIG.device,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
model = PeftModel.from_pretrained(base_model, CONFIG.model_id)
model = model.merge_and_unload()

# Upload

In [7]:
tokenizer.push_to_hub(
    repo_id=CONFIG.repo_id,
    use_temp_dir=False
)
model.push_to_hub(
    repo_id=CONFIG.repo_id,
    use_temp_dir=False
)

README.md:   0%|          | 0.00/8.28k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/PathFinderKR/Waktaverse-Llama-3-KO-8B-Instruct/commit/5bfbd92b09c175942efd041a55498d2705c47f50', commit_message='Upload LlamaForCausalLM', commit_description='', oid='5bfbd92b09c175942efd041a55498d2705c47f50', pr_url=None, pr_revision=None, pr_num=None)